# Efficient Yelp API Calls CORE
Loveida Lucero

In [1]:
# Standard Imports
import numpy as np
import pandas as pd
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
with open('/Users/Lovei/.secret/yelp_api.json') as f:
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [3]:
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
yelp_api

In [4]:
# use our yelp_api variable's search_query method to perform our API call
search_results = yelp_api.search_query(location='Salem, MA',
                                       term='Soup')
print(type(search_results))
search_results.keys()

<class 'dict'>


dict_keys(['businesses', 'total', 'region'])

In [5]:
# set our API call parameters 
LOCATION = 'Salem, MA'
TERM = 'Soup'

In [6]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE2 = "Data/results_in_progress_Salem_Soup.json"
JSON_FILE2

'Data/results_in_progress_Salem_Soup.json'

In [7]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE2)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE2)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE2} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE2,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE2} already exists.")

[i] Data/results_in_progress_Salem_Soup.json already exists.


In [8]:
## Load previous results and use len of results for offset
with open(JSON_FILE2,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


In [9]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [10]:
## How many results total?
total_results = results['total']
total_results

317

In [11]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [12]:
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

16

In [13]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE2,'w') as f:
     json.dump(previous_results,f)

In [14]:
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 

  0%|          | 0/16 [00:00<?, ?it/s]

In [15]:
## delete file and confirm it no longer exits.
os.remove(JSON_FILE2)
os.path.isfile(JSON_FILE2)

False

In [16]:
def create_json_file(JSON_FILE2,  delete_if_exists=False):
    
    ## Check if JSON_FILE2 exists
    file_exists = os.path.isfile(JSON_FILE2)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE2} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE2)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE2,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE2} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE2} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE2)
        
        ## If JSON_FILE2 included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE2,'w') as f:
            json.dump([],f)  

In [17]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE2, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE2,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[i] Data/results_in_progress_Salem_Soup.json not found. Saving empty list to new file.
- 0 previous results found.


16

In [18]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE2, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE2,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/16 [00:00<?, ?it/s]

In [20]:
# load final results
final_df = pd.read_json(JSON_FILE2)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,RwtYEWrcSOiCouIytzhqbA,new-england-soup-factory-salem,New England Soup Factory,https://s3-media2.fl.yelpcdn.com/bphoto/UBi4MC...,False,https://www.yelp.com/biz/new-england-soup-fact...,234,"[{'alias': 'tradamerican', 'title': 'American'...",4.0,"{'latitude': 42.52053, 'longitude': -70.89577}","[delivery, pickup]",$,"{'address1': '140 Washington St', 'address2': ...",+19787440444,(978) 744-0444,561.663115
1,k1ROSfBsw9DPGwvYeFtr7w,kokeshi-salem,Kokeshi,https://s3-media4.fl.yelpcdn.com/bphoto/5mAGUV...,False,https://www.yelp.com/biz/kokeshi-salem?adjust_...,222,"[{'alias': 'asianfusion', 'title': 'Asian Fusi...",4.0,"{'latitude': 42.52062057489013, 'longitude': -...",[delivery],$$,"{'address1': '288 Derby St', 'address2': '', '...",+19785944197,(978) 594-4197,810.885887
2,dzj7ljw37--iE-IMjNizBQ,chezcasa-salem,ChezCasa,https://s3-media2.fl.yelpcdn.com/bphoto/1xheVz...,False,https://www.yelp.com/biz/chezcasa-salem?adjust...,66,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",5.0,"{'latitude': 42.52927, 'longitude': -70.88961}",[delivery],NaN,"{'address1': '97 Bridge St', 'address2': '', '...",+19787442143,(978) 744-2143,1636.493230
3,Qr0MnUz8IKt8_1lKvXz8mw,gulu-gulu-cafe-salem,Gulu Gulu Cafe,https://s3-media3.fl.yelpcdn.com/bphoto/v_NfOD...,False,https://www.yelp.com/biz/gulu-gulu-cafe-salem?...,1078,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",4.0,"{'latitude': 42.521312, 'longitude': -70.896277}","[delivery, pickup]",$$,"{'address1': '247 Essex St', 'address2': '', '...",+19787408882,(978) 740-8882,599.475465
4,IRgsYIVjSZ0u2nm9gEqy5A,ray-adeas-mediterranean-kitchen-salem,Ray Adea's Mediterranean Kitchen,https://s3-media4.fl.yelpcdn.com/bphoto/Yf34QJ...,False,https://www.yelp.com/biz/ray-adeas-mediterrane...,171,"[{'alias': 'mediterranean', 'title': 'Mediterr...",5.0,"{'latitude': 42.51853, 'longitude': -70.89375}","[delivery, pickup]",$,"{'address1': '90 Lafayette St', 'address2': ''...",+19785944271,(978) 594-4271,558.918737


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
312,xuW6SviETDtiYSUzLQVUyA,prime-roast-beef-and-pizza-danvers,Prime Roast Beef & Pizza,https://s3-media3.fl.yelpcdn.com/bphoto/8tKXPY...,False,https://www.yelp.com/biz/prime-roast-beef-and-...,130,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",3.5,"{'latitude': 42.5539024525685, 'longitude': -7...","[pickup, delivery]",$$,"{'address1': '85 Andover St', 'address2': '', ...",+19787679505,(978) 767-9505,6556.385061
313,Hj5qmsYC35Kw1mdmqlDvHQ,qdoba-mexican-eats-peabody,QDOBA Mexican Eats,https://s3-media2.fl.yelpcdn.com/bphoto/k6YSlt...,False,https://www.yelp.com/biz/qdoba-mexican-eats-pe...,79,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",3.0,"{'latitude': 42.547345812634, 'longitude': -70...","[pickup, delivery]",$,"{'address1': '229 Andover St', 'address2': '',...",+19788186232,(978) 818-6232,5163.800316
314,xtum8zS3BO9E1qcUOpi96w,rizzos-roast-beef-and-pizza-peabody-peabody,Rizzo's Roast Beef & Pizza,https://s3-media3.fl.yelpcdn.com/bphoto/yd8J2I...,False,https://www.yelp.com/biz/rizzos-roast-beef-and...,72,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",2.5,"{'latitude': 42.5055666, 'longitude': -70.9516...","[pickup, delivery]",$$,"{'address1': '178 Lynn St', 'address2': '', 'a...",+19785312788,(978) 531-2788,4417.713382
315,N1N0AtgiSf0thGPEIVtlCA,marano-s-trattoria-peabody-3,Marano’s Trattoria,https://s3-media2.fl.yelpcdn.com/bphoto/m8YEqL...,False,https://www.yelp.com/biz/marano-s-trattoria-pe...,73,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",3.5,"{'latitude': 42.55068473751309, 'longitude': -...",[delivery],$,"{'address1': '637 Lowell St', 'address2': '', ...",+19785350411,(978) 535-0411,9168.112979
316,CvQF7p0Hw3LWPIcEx_CdGw,omega-pizza-middleton,Omega Pizza,https://s3-media3.fl.yelpcdn.com/bphoto/RiBuxm...,False,https://www.yelp.com/biz/omega-pizza-middleton...,64,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",3.5,"{'latitude': 42.5771205, 'longitude': -70.9973...","[pickup, delivery]",$,"{'address1': '232 S Main St', 'address2': '', ...",+19787773900,(978) 777-3900,10503.919983


In [21]:
# check for duplicate ID's 
final_df.duplicated(subset='id').sum()

0

In [23]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_Salem_Soup.csv.gz', compression='gzip',index=False)